In [1]:
from poppy.creatures import PoppyHumanoid
import numpy as np
import cv2

# own modules
import goals
import poppy_qnetwork as q_network
import vision


ARM_LENGTH_1 = 15.1
ARM_LENGTH_2 = 10.1
ARM_OFFSET = 10.95
ANGULAR_ARM_VELOCITY = 1
GOAL_THRESHOLD = 0.5
MAX_STEPS = 500
NUM_OF_STATES = 4
SIDE = 'l' # 'r'

JOINT_LIMITS = ((-20., 195.), (-148., 1.)) # LEFT ARM
#JOINT_LIMITS = ((-195., 20.), (-1., 148.)) # RIGHT ARM 

In [25]:
def set_action(poppy, action):
    waiting = False
    if action==0:
        poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position - ANGULAR_ARM_VELOCITY, 0, wait=waiting)
    elif action==1:
        poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position + ANGULAR_ARM_VELOCITY, 0, wait=waiting) 
    elif action==2:
        poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position - ANGULAR_ARM_VELOCITY, 0, wait=waiting)
    elif action==3:
        poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position + ANGULAR_ARM_VELOCITY, 0, wait=waiting)
    return

def get_poppy_angles(poppy):
    # normalize thetas to [-1.0, +1.0]
    theta1 = float(poppy.l_shoulder_x.present_position)
    theta2 = float(poppy.l_elbow_y.present_position)
    return np.array([theta1, theta2])

def get_poppy_normed_angles(poppy):
    # normalize thetas to [-1.0, +1.0]
    theta = get_poppy_angles(poppy)

    range1_half = 0.5*(JOINT_LIMITS[0][1] - JOINT_LIMITS[0][0])
    theta1 = (theta[0] - range1_half)/range1_half

    range2_half = 0.5*(JOINT_LIMITS[1][1] - JOINT_LIMITS[1][0])
    theta2 = (theta[1] - range2_half)/range2_half        
    return np.array([theta1, theta2])

def get_poppy_position(poppy):
    # relative position of hand with respect to shoulder
    #x,y = np.array(poppy.get_object_position(SIDE+'_forearm_visual')[0:2]) - np.array(poppy.get_object_position(SIDE+'_shoulder_x')[0:2])
    #return np.array([-y,x])*100.0

    # Forward Kinematics
    theta = get_poppy_angles(poppy)*np.pi/180.
    
    pos = np.array([0.0, 0.0])
    pos[0] = ARM_LENGTH_1*np.cos(theta[0]) + ARM_LENGTH_2*np.cos(theta[0]+theta[1])
    pos[1] = ARM_LENGTH_1*np.sin(theta[0]) + ARM_LENGTH_2*np.sin(theta[0]+theta[1])
    return pos

def get_4_state(poppy, goal_pos):
    # normalized [dx, dy]
    dist = (goal_pos - get_poppy_position(poppy)) / (ARM_LENGTH_1+ARM_LENGTH_2)

    # normalized [theta_1, theta_2]
    theta = get_poppy_normed_angles(poppy)

    # state = [dx,dy,theta1,theta2]
    return np.array([dist, theta])

def episode_finished(poppy, goal_pos):
    agent_pos = get_poppy_position(poppy)
    distance = np.linalg.norm(agent_pos - goal_pos)
    if distance < GOAL_THRESHOLD:
        return True # episode finished if agent already at goal
    else:
        return False

In [3]:
# load qnetwork
network = q_network.QNetwork()

# init poppy
poppy = PoppyHumanoid()

# init camera
#camera = cv2.VideoCapture(0)

In [29]:
# DE-activate motors
L_MOTORS = [poppy.l_shoulder_x, poppy.l_shoulder_y, poppy.l_elbow_y, poppy.l_arm_z]
R_MOTORS = [poppy.r_shoulder_x, poppy.r_shoulder_y, poppy.r_elbow_y, poppy.r_arm_z]
for m in L_MOTORS+R_MOTORS+[poppy.bust_y, poppy.bust_x]:
    m.compliant = True

In [30]:
# activate motors
L_MOTORS = [poppy.l_shoulder_x, poppy.l_shoulder_y, poppy.l_elbow_y, poppy.l_arm_z]
R_MOTORS = [poppy.r_shoulder_x, poppy.r_shoulder_y, poppy.r_elbow_y, poppy.r_arm_z]
for m in L_MOTORS:
    m.compliant = False

# move to initial position
poppy.l_shoulder_y.goto_position(-90,1,wait=True)
poppy.l_arm_z.goto_position(-90,1,wait=True)
poppy.r_shoulder_y.goto_position(-90,1,wait=True)
poppy.r_arm_z.goto_position(90,1,wait=True)

# some starting position
poppy.l_elbow_y.goto_position(-90,1,wait=True)
poppy.l_shoulder_x.goto_position(60, 1, wait=True)

In [6]:
# set initial goal position
#goal_pos = np.array([0.,0.])
#print 'WARNING: set initial goal to a reachable position!'

# simulated GOALS
goal = goals.Goal_Arm(ARM_LENGTH_1, ARM_LENGTH_2)

In [32]:
for _ in range(500): #while True:
    # get goal (VISION)
    #_, image = camera.read()
    #tmp_goal = vision.getGoalPosition(image)
    #if tmp_goal: # only update goal if a goal could be detected
    #    goal_pos = tmp_goal
    #cv2.imshow('image', image)
    #cv2.waitKey(1)

    # get goal (simulated GOALS)
    goal_pos = np.array([20., 10.]) 
    #goal_pos = goal.get_position()

    # get state
    state = get_4_state(poppy, goal_pos)

    # stop actions when goal reached
    if not episode_finished(poppy, goal_pos):
        q = network.predict(state.reshape(1, NUM_OF_STATES).tolist()[0])
        action = np.argmax(q) # choose best action from Q(s,a)
        # action
        set_action(poppy, action)
    else:
        print 'reached goal position'
        
    #print "action=%s | goal=%s | pos=%s | state=%s" % (action, goal_pos, get_poppy_position(poppy), state)
    

reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached goal position
reached go

In [ ]:
cv2.destroyAllWindows()
camera.release()
poppy.close()